# Clean addresses in 311 no heat complaints

Describe what this notebook does TK

In [17]:
# import packages
import pandas as pd
import re

In [108]:
# ignore chained assignment warning messages
pd.options.mode.chained_assignment = None

In [110]:
# import data
df = pd.read_csv('../data/source/311_Heat_Complaints.csv')

In [111]:
df.tail()

,SR Number,Date/Time Opened,Service Request Address,Service Request Status,Answer
16574,SR24-00139193,01/23/24 07:37 PM,10437 S HALE AVE<br>Chicago Illinois 60643,Open,NaN
16575,SR24-00139626,01/23/24 09:56 PM,860 N DEWITT PL<br>Apt 1502<br>Chicago Illinoi...,Completed,NaN
16576,SR24-00141986,01/24/24 10:14 AM,7311 S EAST END AVE<br>Chicago Illinois 60649,Open,NaN
16577,SR24-00142172,01/24/24 10:30 AM,611 W BRIAR PL<br>103<br>Chicago Illinois 60657,Open,NaN
16578,Total,16578,NaN,NaN,NaN


In [112]:
# delete the total row
df.drop(16578, inplace=True)

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16578 entries, 0 to 16577
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   SR Number                16578 non-null  object
 1   Date/Time Opened         16578 non-null  object
 2   Service Request Address  16573 non-null  object
 3   Service Request Status   16578 non-null  object
 4   Answer                   15028 non-null  object
dtypes: object(5)
memory usage: 647.7+ KB


## Parse zipcode from the address column

Splits the service request address by the br tag to isolate a clean street address portion and then uses regex to find a five-digit zipcode pattern in the remaining address.Creates a new zipcode column with the five digit zipcode.

In [114]:
# split Service Request Address by <br> into max three columns
df[['street_address', 'address_1', 'address_2']] = df['Service Request Address'].str.split('<br>',expand=True)

In [152]:
# replace None with an empty string
df.fillna('',inplace=True)

In [153]:
df.tail()

,SR Number,Date/Time Opened,Service Request Address,Service Request Status,Answer,street_address,address_1,address_2,zipcode,apt_num
16573,SR24-00139103,01/23/24 07:13 PM,860 N DEWITT PL<br>1704<br>Chicago Illinois 60611,Completed,,860 N DEWITT PL,1704,Chicago Illinois 60611,60611,1704 Chicago Illinois 60611
16574,SR24-00139193,01/23/24 07:37 PM,10437 S HALE AVE<br>Chicago Illinois 60643,Open,,10437 S HALE AVE,Chicago Illinois 60643,,60643,
16575,SR24-00139626,01/23/24 09:56 PM,860 N DEWITT PL<br>Apt 1502<br>Chicago Illinoi...,Completed,,860 N DEWITT PL,Apt 1502,Chicago Illinois 60611,60611,Apt 1502 Chicago Illinois 60611
16576,SR24-00141986,01/24/24 10:14 AM,7311 S EAST END AVE<br>Chicago Illinois 60649,Open,,7311 S EAST END AVE,Chicago Illinois 60649,,60649,
16577,SR24-00142172,01/24/24 10:30 AM,611 W BRIAR PL<br>103<br>Chicago Illinois 60657,Open,,611 W BRIAR PL,103,Chicago Illinois 60657,60657,103 Chicago Illinois 60657


In [154]:
# parse address_1 and address_2 for a five digit zip code and put into a new column called zipcode

pattern = r'\b\d{5}\b'

def get_zip(value):
    zipcode = re.findall(pattern, str(value)) # returns a list of all matches
    if len(zipcode)>0: # if list contains zipcode
        return zipcode[0] # return first value in list
    else:
        return ''

In [155]:
# create a new column for zipcode
df['zipcode'] = ''

# loop through each row to parse zipcode first in address_1 and if not found there, address_2
for index, row in df.iterrows():
    zipcode = get_zip(row['address_1'])
    row['zipcode'] = zipcode
    if zipcode == '':
        zipcode = get_zip(row['address_2'])
        row['zipcode'] = zipcode

## Parse apartment number from the address column

Combine address_1 and address_2 to get all the non street address info into one column. Then did find and replace for 'Chicago Illinois' and a five digit zipcode to isolate the apartment number in a new column called apt_num.

In [156]:
# create a column for apartment number that combines address_1 and address_2
df['apt_num'] = df['address_1'] + ' ' + df['address_2']

In [158]:
df.tail()

,SR Number,Date/Time Opened,Service Request Address,Service Request Status,Answer,street_address,address_1,address_2,zipcode,apt_num
16573,SR24-00139103,01/23/24 07:13 PM,860 N DEWITT PL<br>1704<br>Chicago Illinois 60611,Completed,,860 N DEWITT PL,1704,Chicago Illinois 60611,60611,1704 Chicago Illinois 60611
16574,SR24-00139193,01/23/24 07:37 PM,10437 S HALE AVE<br>Chicago Illinois 60643,Open,,10437 S HALE AVE,Chicago Illinois 60643,,60643,Chicago Illinois 60643
16575,SR24-00139626,01/23/24 09:56 PM,860 N DEWITT PL<br>Apt 1502<br>Chicago Illinoi...,Completed,,860 N DEWITT PL,Apt 1502,Chicago Illinois 60611,60611,Apt 1502 Chicago Illinois 60611
16576,SR24-00141986,01/24/24 10:14 AM,7311 S EAST END AVE<br>Chicago Illinois 60649,Open,,7311 S EAST END AVE,Chicago Illinois 60649,,60649,Chicago Illinois 60649
16577,SR24-00142172,01/24/24 10:30 AM,611 W BRIAR PL<br>103<br>Chicago Illinois 60657,Open,,611 W BRIAR PL,103,Chicago Illinois 60657,60657,103 Chicago Illinois 60657


In [161]:
# find and replace Chicago Illinois substrings
city_pattern = r'Chicago Illinois\b'

df['apt_num'] = df['apt_num'].str.replace(pattern, '')

# find and replace zipcode patterns
pattern = r'\b\d{5}\b'

df['apt_num'] = df['apt_num'].str.replace(pattern, '')

/tmp/ipykernel_38724/400239471.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['apt_num'] = df['apt_num'].str.replace(pattern, '')
/tmp/ipykernel_38724/400239471.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['apt_num'] = df['apt_num'].str.replace(pattern, '')


In [163]:
df.tail()

,SR Number,Date/Time Opened,Service Request Address,Service Request Status,Answer,street_address,address_1,address_2,zipcode,apt_num
16573,SR24-00139103,01/23/24 07:13 PM,860 N DEWITT PL<br>1704<br>Chicago Illinois 60611,Completed,,860 N DEWITT PL,1704,Chicago Illinois 60611,60611,1704
16574,SR24-00139193,01/23/24 07:37 PM,10437 S HALE AVE<br>Chicago Illinois 60643,Open,,10437 S HALE AVE,Chicago Illinois 60643,,60643,
16575,SR24-00139626,01/23/24 09:56 PM,860 N DEWITT PL<br>Apt 1502<br>Chicago Illinoi...,Completed,,860 N DEWITT PL,Apt 1502,Chicago Illinois 60611,60611,Apt 1502
16576,SR24-00141986,01/24/24 10:14 AM,7311 S EAST END AVE<br>Chicago Illinois 60649,Open,,7311 S EAST END AVE,Chicago Illinois 60649,,60649,
16577,SR24-00142172,01/24/24 10:30 AM,611 W BRIAR PL<br>103<br>Chicago Illinois 60657,Open,,611 W BRIAR PL,103,Chicago Illinois 60657,60657,103


## Export cleaned data as csv

In [166]:
# add a city and state column
df['city'] = 'Chicago'
df['state'] = 'Illinois'

# delete working columns, adddress_1 and address_2
df.drop('address_1', axis=1, inplace=True)
df.drop('address_2', axis=1, inplace=True)

In [170]:
df.to_csv('../data/processed/cleaned_complaints.csv')